In [1]:
# Use the Azure Machine Learning data collector to log various metrics
from azureml.logging import get_azureml_logger
logger = get_azureml_logger()

In [2]:
# Use Azure Machine Learning history magic to control history collection
# History is off by default, options are "on", "off", or "show"
%azureml history on

History logging enabled


In [3]:
from __future__ import print_function
from azure.storage.blob import BlockBlobService
import os
import cntk as C
import numpy as np
import scipy.sparse
import sys
import glob

In [4]:
#minibatch size is size of single minibatch.  To calc optimal size:
#mem = memory - neural net params * float32 size
#size = mem / (num_features * float32 size)
minibatch_size = 10000
#epoch size is virtual concept of whole data set.
epoch_size = 50000
num_features = 212

#sweep is a full pass through the data set.
max_sweeps = 1000

num_outputs = 58

In [5]:
# Define the task.
x = C.input_variable(num_features)
y = C.input_variable(num_outputs, is_sparse=True)
base_container = 'data-files'
train_folder = 'train_ctf'
test_folder = 'test_ctf'
base_local = '/data/bikeshare/'
model_path = os.path.join('.', 'outputs', 'model.dnn')
blob_acct_name = 'kpmgstorage1'
blob_acct_key = '2+BXi305SN45G9yyhykvp7Ij6KYka9W/WvRH4aG5fOuK+9Fenk5Yhg6X6lUMrxjpxE4wKxXyk9NYptzUpZYQkQ=='


In [6]:
final_train_file = base_local + train_folder + "/" + 'train_complete.ctf'
final_test_file = base_local + test_folder + "/" + 'test_complete.ctf'

In [7]:
final_train_file


'/data/bikeshare/train_ctf/train_complete.ctf'

In [8]:
#Download the train data into our docker container
bbs = BlockBlobService(account_name=blob_acct_name, account_key=blob_acct_key)

In [9]:
def download_and_merge_ctf(remote_base, local_base, relative, output_file_name):
    generator = bbs.list_blobs(remote_base)
    for blob in generator:
        tokens = blob.name.split('/')
        if(tokens[0] == relative
          and len(tokens) > 1
          and tokens[1] != "_SUCCESS"):
            if not os.path.exists(local_base + relative):
                os.makedirs(local_base + relative)
            bbs.get_blob_to_path(remote_base, blob.name, local_base + '/' + blob.name + '.ctf')


    read_files = glob.glob(local_base + relative + "/*.ctf")
    with open(output_file_name, "wb") as outfile:
        for f in read_files:
            with open(f, "rb") as infile:
                outfile.write(infile.read())

In [10]:
download_and_merge_ctf(base_container, base_local, train_folder, final_train_file)
#download_and_merge_ctf(base_container, base_local, test_folder, 'test_complete.ctf')

In [11]:
final_train_file

'/data/bikeshare/train_ctf/train_complete.ctf'

In [12]:
# Read a COMPOSITE reader to read data from both the image map and CTF files
def create_minibatch_source(ctf_file, is_training, num_outputs):
    
    # create CTF DESERIALIZER for CTF file
    data_source = C.io.CTFDeserializer(ctf_file, C.io.StreamDefs(
        features = C.io.StreamDef(field="features", shape=num_features),
        label = C.io.StreamDef(field="label", shape=num_outputs)))

    # create a minibatch source by compositing them together 
    return C.io.MinibatchSource([data_source], max_sweeps=max_sweeps, randomize=is_training)

In [13]:
train_source = create_minibatch_source(final_train_file, True, num_outputs)
#test_source = create_minibatch_source(final_test_file, False, num_outputs)

In [14]:
input_map = {    
    x : train_source.streams.features,
    y : train_source.streams.label
}


In [15]:
num_neurons = 100
def create_model(x):
    with C.layers.default_options(init = C.layers.glorot_uniform(), activation = C.relu):            
            h = C.layers.Dense(num_neurons, name="first")(x)
            h = C.layers.Dense(num_neurons, name="second")(h)
            h = C.layers.Dense(num_neurons, name="third")(h)
            h = C.layers.Dropout(dropout_rate=0.5)(h)
            p = C.layers.Dense(num_outputs, activation = None, name="prediction")(h)         
            return p

In [16]:
model = create_model(x)

In [17]:
# loss function
loss = C.cross_entropy_with_softmax(model, y)

In [18]:
#error metric function
error = C.classification_error(model, y)

In [19]:
#combine error and loss for training usage (AKA Criterion function)
criterion = C.combine([loss, error])

In [20]:
lr_schedule = [0.01]*100 + [0.001]*100 + [0.0001]
learner = C.sgd(model.parameters, lr_schedule, epoch_size=epoch_size)

In [21]:
#frequency is defined by number of samples
progress_writer = C.logging.ProgressPrinter(freq=epoch_size)

In [22]:
#frequence is by number of samples
checkpoint_config = C.CheckpointConfig(filename=model_path, frequency=(epoch_size * 10), restore = False)


In [23]:
#test_config = C.TestConfig(test_source)
def custom_cv_func(index, average_error, cv_num_samples, cv_num_minibatches):
    print("CV Error: " + str(average_error))
    logger.log("Error", average_error)
    return True

In [24]:
cv_config = C.CrossValidationConfig(minibatch_source = train_source,
                                    minibatch_size = minibatch_size,
                                    model_inputs_to_streams = input_map,
                                    frequency = epoch_size,
                                    max_samples = epoch_size,
                                    callback=custom_cv_func)


In [25]:
progress = criterion.train(train_source, 
                           minibatch_size = minibatch_size,
                           model_inputs_to_streams = input_map,
                           epoch_size = epoch_size,
                           max_epochs = 100, 
                           parameter_learners = [learner], 
                           callbacks = [progress_writer, cv_config])#, checkpoint_config])#, test_config])


Learning rate per minibatch: 0.01
Finished Epoch[1]: loss = 4.056413 * 50000, metric = 97.92% * 50000 8.304s (6021.2 samples/s);
Finished Evaluation [1]: Minibatch[1-5]: metric = 97.65% * 50000;
CV Error: 0.9765
Finished Epoch[2]: loss = 4.040617 * 50000, metric = 97.08% * 50000 2.424s (20627.1 samples/s);
Finished Evaluation [2]: Minibatch[1-5]: metric = 96.51% * 50000;
CV Error: 0.96512
Finished Epoch[3]: loss = 4.025062 * 50000, metric = 95.73% * 50000 2.118s (23607.2 samples/s);
Finished Evaluation [3]: Minibatch[1-5]: metric = 93.85% * 50000;
CV Error: 0.93854
Finished Epoch[4]: loss = 4.010050 * 50000, metric = 93.71% * 50000 4.031s (12403.9 samples/s);
Finished Evaluation [4]: Minibatch[1-5]: metric = 89.12% * 50000;
CV Error: 0.89122
Finished Epoch[5]: loss = 3.994584 * 50000, metric = 90.66% * 50000 2.724s (18355.4 samples/s);
Finished Evaluation [5]: Minibatch[1-5]: metric = 83.52% * 50000;
CV Error: 0.83516
Finished Epoch[6]: loss = 3.978695 * 50000, metric = 87.18% * 50000 

Finished Epoch[46]: loss = 2.858915 * 50000, metric = 67.57% * 50000 2.530s (19762.8 samples/s);
Finished Evaluation [46]: Minibatch[1-5]: metric = 67.78% * 50000;
CV Error: 0.6778
Finished Epoch[47]: loss = 2.836153 * 50000, metric = 67.86% * 50000 2.198s (22748.0 samples/s);
Finished Evaluation [47]: Minibatch[1-5]: metric = 67.79% * 50000;
CV Error: 0.67794
Finished Epoch[48]: loss = 2.813821 * 50000, metric = 67.82% * 50000 2.250s (22222.2 samples/s);
Finished Evaluation [48]: Minibatch[1-5]: metric = 67.33% * 50000;
CV Error: 0.67328
Finished Epoch[49]: loss = 2.775873 * 50000, metric = 67.43% * 50000 2.346s (21312.9 samples/s);
Finished Evaluation [49]: Minibatch[1-5]: metric = 67.36% * 50000;
CV Error: 0.67358
Finished Epoch[50]: loss = 2.761226 * 50000, metric = 67.50% * 50000 2.919s (17129.2 samples/s);
Finished Evaluation [50]: Minibatch[1-5]: metric = 67.24% * 50000;
CV Error: 0.67242
Finished Epoch[51]: loss = 2.735036 * 50000, metric = 67.37% * 50000 2.842s (17593.2 sample

Finished Evaluation [91]: Minibatch[1-5]: metric = 67.90% * 50000;
CV Error: 0.679
Finished Epoch[92]: loss = 2.413275 * 50000, metric = 69.26% * 50000 2.352s (21258.5 samples/s);
Finished Evaluation [92]: Minibatch[1-5]: metric = 67.40% * 50000;
CV Error: 0.674
Finished Epoch[93]: loss = 2.395472 * 50000, metric = 68.74% * 50000 2.126s (23518.3 samples/s);
Finished Evaluation [93]: Minibatch[1-5]: metric = 67.38% * 50000;
CV Error: 0.67378
Finished Epoch[94]: loss = 2.397295 * 50000, metric = 68.76% * 50000 2.489s (20088.4 samples/s);
Finished Evaluation [94]: Minibatch[1-5]: metric = 67.57% * 50000;
CV Error: 0.67568
Finished Epoch[95]: loss = 2.398835 * 50000, metric = 68.83% * 50000 2.315s (21598.3 samples/s);
Finished Evaluation [95]: Minibatch[1-5]: metric = 67.59% * 50000;
CV Error: 0.67588
Finished Epoch[96]: loss = 2.390077 * 50000, metric = 68.82% * 50000 2.330s (21459.2 samples/s);
Finished Evaluation [96]: Minibatch[1-5]: metric = 67.13% * 50000;
CV Error: 0.67134
Finished 

In [26]:
inf_model = C.softmax(model)

In [27]:
inf_model.save(model_path)

In [28]:
model_path

'.\\outputs\\model.dnn'